In [6]:
from cdapython import Q, columns, unique_terms, query
import cdapython
import pandas as pd 
print(cdapython.__version__)
Q.set_host_url("http://35.192.60.10:8080/")

2022.6.9


In [7]:
import yaml
from yaml import Loader
from collections import defaultdict
class data_query_validation:
    def __init__(self, mapping_yaml) -> None:
        # map of field name -> set of values
        self.mapping_yaml = mapping_yaml
        self.entity_fields = self._init_entity_fields()
    def _init_entity_fields(self):
        return_dict = {}
        MandT = yaml.load(open(self.mapping_yaml, "r"), Loader=Loader)
        for entity, MorT_dict in MandT.items():
            return_dict[entity.replace('_merge','')] = MorT_dict['Mapping']
        return return_dict
    def two_matching_recs(self, recs, entity_name):
        diffs = {}
        same = True
        rec_sources = list(recs.keys())
        for field in self.entity_fields[entity_name]:
            rec_compare = [recs[source].get(field) for source in rec_sources]
            #type_of_compare = ''
            for i in rec_compare:
                if isinstance(rec_compare[0],list) and isinstance(rec_compare[1],list):
                    if not isinstance(rec_compare[0][0],dict):
                        rec_compare[0].sort()
                        rec_compare[1].sort()
                if rec_compare[0] != rec_compare[1]:
                    if isinstance(rec_compare[1],int) and isinstance(rec_compare[0], str):
                        if int(rec_compare[0])== rec_compare[1]:
                            continue
                    if None not in rec_compare:
                        same = False
                        diffs[field] = {rec_sources[0]:rec_compare[0],
                                        rec_sources[1]:rec_compare[1]}
        return same, diffs
    def all_identical_dups(self, results_dict, entity_name):
        # determines if all duplicate entities are identical
        all_identical = True
        all_diffs = defaultdict(list)
        for idd, recs in results_dict.items():
            for i in range(1, len(recs)):
                same, diffs = self.two_matching_recs({0:recs[0], i:recs[i]}, entity_name)
                if not same:
                    all_identical = False
                    all_diffs[idd].append(diffs)
                else:
                    all_diffs[idd].append(None)
        return all_identical, all_diffs
                    
            
    def two_matching_queries(self, data_file_results, query_results, entity):
        df_ids = set([i['id'] for i in data_file_results])
        qr_ids = set([i['id'] for i in query_results])
        return_same_ids = len(df_ids) == len(qr_ids)
        return_same_num_recs = len(data_file_results)==len(query_results)
        dups_in_data_return = len(df_ids) == len(data_file_results)
        dups_in_query_return = len(qr_ids) == len(query_results)
        no_dups = dups_in_data_return and dups_in_query_return
        missing_ids = []
        added_ids = []
        ids_w_diff_data = []
        #Simple comparison - same ids and number of entities returned
        if return_same_num_recs and return_same_ids and no_dups:
            df_recs = {}
            qr_recs = {}
            for idd in df_ids:
                for rec in data_file_results:
                    if rec['id'] == idd:
                        df_recs[idd] = rec
                        #print('id: ' + idd)
                        #print('age_at_diagnosis: ' + str(rec['age_at_diagnosis']))
                        #print(type(rec['age_at_diagnosis']))
                        break
                for rec in query_results:
                    if rec['id'] == idd:
                        qr_recs[idd] = rec
                        break
                        
                match, diffs = self.two_matching_recs({'query1': df_recs[idd], 
                                                       'query2': qr_recs[idd]}, entity)
                if not match:
                    print(idd)
                    print(diffs)
        #elif len(data_file_results) != len(query_results) and df_ids == qr_ids:
            # There must be duplicates if distinct ids are the same.
            # Reduce dups in longer file - check that data is the same for the dups
            # If not then need to report that error as well.
        else:
            print('fucked')
        return None
    
    def count_all_entities(self, q, print_dfs = False):
        q_dict = {'Subjects': q.subject,
                  'ResearchSubjects': q.researchsubject,
                  'Specimens': q.specimen,
                  'Diagnoses': q.diagnosis,
                  'Treatments': q.treatment
                 }
        q_return = {}
        for entity_name, q_call in q_dict.items():
            q_return[entity_name]= q_call.count.run()
        
        if print_dfs:
            for entity_name, q_call in q_return.items():
                print(entity_name + ': ')
                print(q_call)
        return q_return
        
    def all_q_entities(self, q):
        q_dict = {'Subjects': q.subject,
                  'ResearchSubjects': q.researchsubject,
                  'Specimens': q.specimen,
                  'Diagnoses': q.diagnosis,
                  'Treatments': q.treatment
                 }
        q_return = defaultdict(list)
        for entity_name, q_call in q_dict.items():
            for page in q_call.run().paginator():
                q_return[entity_name].extend(page)
            print(entity_name + ": " + str(len(q_return[entity_name])))
        return q_return
    
    def all_q_files(self, q):
        qresults = defaultdict(list)
        for page in q.subject.file.run().paginator():
            qresults['Subject_Files'].extend(page)
        for page in q.researchsubject.file.run().paginator():
            qresults['ResearchSubject_Files'].extend(page)
        for page in q.specimen.file.run().paginator():
            qresults['Specimen_Files'].extend(page)
        return qresults

In [3]:
dqv = data_query_validation("~/")

NameError: name 'all_q_entities' is not defined